In [1]:
import pandas as pd
import numpy as np
import json
from gensim.models import KeyedVectors

In [2]:
%run constants.ipynb
%run functions.ipynb
%run dictionnaries.ipynb

In [3]:
model_w2v = KeyedVectors.load_word2vec_format(pathword2vec, binary=True)

In [4]:
prevoc = get_prevoc()
voc = get_voc(model_w2v, prevoc)

tabs = ['default','bars']
cate = ['Seat', 'Food']
manu = ['Airbus', 'Boeing', 'Cessna', 'Cirrus ','Pilatus', 'Matra']
airl = list(df_airline['airline'].apply(preprocess))
airc = list(df_aircraft['model'].apply(preprocess))

##### make the database

In [5]:
bdd = make_bdd(prevoc, 50)
bdd.head()

,session,date,json
0,0,19/06/2010 09h14m32s,"{""tab"": ""Food"", ""filters"": {""manufacturer"": [""..."
1,0,17/05/2014 16h18m25s,"{""tab"": ""Seat"", ""filters"": {""manufacturer"": [""..."
2,0,21/01/2011 23h06m15s,"{""tab"": ""Food"", ""filters"": {""manufacturer"": [""..."
3,0,09/03/2019 06h39m16s,"{""tab"": ""Seat"", ""filters"": {""manufacturer"": [""..."
4,1,12/06/2012 09h51m31s,"{""tab"": ""default"", ""filters"": {""manufacturer"":..."


##### hash the jsons : create a single representation for every corresponding json

In [6]:
bdd[CT_bdd_hash] = bdd[CT_bdd_json].apply(json_string_to_hash)

##### extract the states

In [7]:
etats = dict([(a,b) for (b,a) in enumerate(sorted(bdd[CT_bdd_hash].unique()))])
id_to_etats = dict((b,a) for (a,b) in etats.items())

##### extract the transitions

In [8]:
transitions = np.zeros([len(etats),len(etats)])
for session in bdd["session"].unique():
    for i in bdd[bdd["session"]==session].index[1:]:
        transitions[etats[bdd[CT_bdd_hash][i-1]], etats[bdd[CT_bdd_hash][i]]] += 1
        
# to proba matrix
transitions = (transitions.T / np.sum(transitions,axis=1)).T
df_transitions = pd.DataFrame(transitions, columns=etats, index=etats)

/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


##### make a prediction

In [9]:
a = id_to_etats[0]

predict_next_state(a, df_transitions)

'{"filters": {"manufacturer": ["airbus", "boeing", "cirrus "], "aircraft": ["40", "404", "c_2_greyhound", "cornell"], "company": ["air_tahiti", "mandarin_airlines", "waltair"], "country": ["solomon_islands"], "category": ["business"], "date": ["15112012", "29022015"]}, "tab": "seat"}'

In [11]:
df_transitions.to_csv(data_directory+'df_transitions.csv', sep='§', encoding='utf-8')